## Import libraries

In [2]:
import pandas as pd #basic data operations 
import numpy as np
import re #for using regex functions
import seaborn as sns
import matplotlib.pyplot as plt # for visualization of data
from matplotlib import style   # to set style for the plots
style.use('ggplot') # ggplot is used here 
from textblob import TextBlob  # to process textual data
from nltk.tokenize import word_tokenize  # to perform tokenization
from nltk.stem import PorterStemmer # to perform stemming 
from nltk.corpus import stopwords #to remove the stopwords
stop_words = set(stopwords.words('english')) #setting stopwords as english
 #different evaluation model for evaluation
import joblib
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings("ignore")

## Load saved models

In [3]:
logreg_model = joblib.load('Trained Models_5k/logregr_model.pkl')
nb_model = joblib.load('Trained Models_5k/nb_model.pkl')
sv_model = joblib.load('Trained Models_5k/svm_model.pkl')
kn_model = joblib.load('Trained Models_5k/kn_model.pkl')
dtc_model = joblib.load('Trained Models_5k/dtc_model.pkl')
rf_model = joblib.load('Trained Models_5k/rf_model.pkl')

In [81]:
X = pd.read_csv('5kDataset-trained.csv')
X_copy = X


In [82]:
X.iloc[0]

dear            0
arjun           0
kumar           0
patidar         0
successfully    0
               ..
rig             0
preferred       0
filipinos       0
visayas         0
mindanao        0
Name: 0, Length: 11732, dtype: int64

In [83]:
X = X.iloc[0] * 0
X = pd.DataFrame(X, columns=X_copy.columns)
# new_row = pd.DataFrame([[int(0)]*len(X_copy.columns)], columns=X_copy.columns)
# X = X.append(new_row, ignore_index=True)
X

,dear,arjun,kumar,patidar,successfully,vaccinated,1st,dose,covishield,07062021,...,surgery,004635,saki,southeast,asi,rig,preferred,filipinos,visayas,mindanao


## Prediction based on user input

Take user input, and store predicted value by each model in the `res` dataframe.

Example input:

`
For all the women and healthcare providers who have been asking about the safety of the #PfizerBioNTech https://t.co/ow0Pglkwte
`

`
New statistics show that Covid-related deaths have reached an all-time high in the past week, with hospitals across the country struggling to keep up with the influx of patients. Many families are now mourning the loss of loved ones, and healthcare workers continue to work tirelessly in difficult conditions.

`

`Vaccines have arrived in #AB .  My Mum has been in isolation for 10 days. 18 residents sick.  2 have passed away inâ€¦ https://t.co/7IzvrleXPc`

In [1]:
res = pd.read_csv("model_results_5k.csv")
user_input = input("Enter a tweet: ")

NameError: name 'pd' is not defined

In [48]:
def data_processing(text):
    text = text.lower()  #lowecase conversion
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)  #removal of URL in our tweets
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text) #removing the stop words
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [49]:
user_input = data_processing(user_input) #applying the function processing

In [50]:
stemmer = PorterStemmer() #doing porterstemmer on the data
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [51]:
user_input = stemming(user_input)
user_input

'vaccines arrived ab mum isolation 10 days 18 residents sick 2 passed away inâ'

In [62]:
vect = CountVectorizer()
user_cv = vect.fit_transform([user_input])
abc = dict(sorted(vect.vocabulary_.items()))
feature_names = list(abc.keys())
feature_names

['10',
 '18',
 'ab',
 'arrived',
 'away',
 'days',
 'inâ',
 'isolation',
 'mum',
 'passed',
 'residents',
 'sick',
 'vaccines']

In [63]:
user_df = pd.DataFrame(user_cv.toarray(), columns=feature_names)
user_df

,10,18,ab,arrived,away,days,inâ,isolation,mum,passed,residents,sick,vaccines
0,1,1,1,1,1,1,1,1,1,1,1,1,1


# Verification

In [64]:
def polarity(text):   # function to calculate the text polarity in our data using textblob
    return TextBlob(text).sentiment.polarity

In [65]:
polarity(user_input)

-0.7142857142857143

In [66]:
X.values

array([[0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [84]:
# Get the column names of X and user_df
X_columns = X.columns
user_columns = user_df.columns

for column in user_columns:
    X.loc[0, column] = 0
    X.fillna(0, inplace=True)
    
    if column not in X_columns:
        user_df = user_df.drop(column, axis=1)
    else:
        X[column] = user_df[column].values
        
user_sparse = csr_matrix(X.values) #training same as testing
user_sparse

<1x11732 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

### sv_model.predict(user_in)

In [85]:
res["predicted"] = [sv_model.predict(user_sparse)[0], nb_model.predict(user_sparse)[0], logreg_model.predict(user_sparse)[0], kn_model.predict(user_sparse)[0], dtc_model.predict(user_sparse)[0], rf_model.predict(user_sparse)[0]]
res

,models,accuracy,predicted
0,SVM,87.0,Neutral
1,Decision Tree,85.7,Negative
2,Random Forest,85.3,Neutral
3,Logistic Regression,84.0,Neutral
4,Naive Bayes,68.5,Neutral
5,KNN,67.5,Neutral
